In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

1. Постройте модель, которая оценивает вероятность того, что в следующем году в африканской стране Central African Republic произойдет кризис на базе 5-летней истории (наличие кризиса, информация о валюте и внешнем долге)

2. Сколько ввп в среднем у стран с наименьшим количеством кризисов в спокойные годы?

3. В какой африканской стране кризисы происходят чаще?

### I. Построим модель, оценивающую вероятность инфляционного кризиса в ЦАР

Предподготовка

1. Загрузим базу данных
2. Выберем интересующую нас страну - Центрально-африканскую республику
3. Немножко пошаманим с переменными

In [2]:
data = pd.read_csv("african_crises.csv", index_col = 0)

In [3]:
data1 = data[data["country"] == "Central African Republic"]

In [4]:
c = "inflation_crises"
e = "exch_usd"

y_raw = data1[c].values

In [5]:
n = 0
M = []
while n != y_raw.shape[0]-5:
    M.append(y_raw[n+5])
    n += 1

In [6]:
M = np.array(M)

In [7]:
x_raw = data1[e].values

In [8]:
for i in range(len(x_raw)):
    x_raw[i] = float(x_raw[i])
x_pre = x_raw/100

In [9]:
i = 0
X = []
Y = []
while i != x_pre.shape[0]-5:
    X.append(x_pre[i:i+5])
    i += 1

In [10]:
X = np.stack(X)

    Переход к тренировке

In [11]:
xtr, xte, mtr, mte = train_test_split(X, M)

In [12]:
model1 = LinearRegression()

In [13]:
model1.fit(xtr, mtr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
print("mse =", mean_squared_error(mte, model1.predict(xte)), 
      "mae = ", mean_absolute_error(mte, model1.predict(xte)))


mse = 0.004826835535310173 mae =  0.05615335868009722


Как видим, ошибка небольшая

In [15]:
X[-1]

array([4.55335971, 4.90912289, 5.06961125, 4.97163104, 4.7564136 ])

In [16]:
M[-1]

0

Попробуем предсказать

In [17]:
model1.predict([X[-1]])

array([0.01273815])

Таким образом, модель предсказывает, что кризиса не будет, и оказывается права

### II. Посчитаем средний ВВП у стран с наименьшим количеством кризисов в спокойные годы

1. Отберём только бескризисные года (отбор по банковскому кризису) и найдём страны, у которых хоть раз не было кризиса
2. Найдём 3 страны, у которых больше всего лет без кризиса
3. Теперь выведем средний ВВП этих стран

In [18]:
data2 = data[data['banking_crisis'] == "no_crisis"]
for i in range(1939, 1946):
    data2 = data2[data2['year'] != i]
for i in range(1939, 1946):
    data2 = data2[data2['year'] != i]

cat, cnt = np.unique(data2["country"].values, return_counts = True)

In [19]:
l = [int(i) for i in cnt]
m = []

for i in range(3):
    m.append(l.index(max(l)))
    l[l.index(max(l))] = 0
    
for i in range(len(m)):
    d1 = data[data['country'] == cat[m[i]]]
    m1 = sum(d1["gdp_weighted_default"].values)
    m2 = d1.shape[0]
    print('in', cat[m[i]], 'the average debt in default vis-a-vis the GDP is', m1/m2)

in Egypt the average debt in default vis-a-vis the GDP is 0.015483870967741935
in South Africa the average debt in default vis-a-vis the GDP is 0.015789473684210523
in Algeria the average debt in default vis-a-vis the GDP is 0.01623529411764706


Как видим, реже всего банковские кризисы происходили в Египте, ЮАР и Алжире

### III. Теперь посмотрим, в какой стране чаще происходят кризисы. Для разнообразия исследуем валютные кризисы

In [20]:
data3 = data[data['currency_crises'] == 1]
cal, cnl = np.unique(data3['country'].values, return_counts = True)
k = [int(i) for i in cnl]
print("Most often, crises occurred in", cal[k.index(max(k))])

Most often, crises occurred in Zimbabwe


Как видим, в отношении валюты среди африканских стран наиболее нестабильна Республика Зимбабве